Startup stuff, sourced from `startup.ipy`

In [1]:
# %load startup.ipy
#! /usr/bin/env python3

import sys
sys.path.append('./python')

import logging.config
import os

import xbx.database as xbxdb
import xbx.util as xbxu
import xbx.config as xbxc
import xbx.build as xbxb
import xbx.run as xbxr

logging.config.fileConfig("logging.ini", disable_existing_loggers=False)

CONFIG_PATH="config.ini"

xbxdb.init(xbxu.get_db_path(CONFIG_PATH))
config = xbxc.Config(CONFIG_PATH)

dbsession = xbxdb.scoped_session()


List RunSessions, ordered by descending timestamp

In [2]:
s=dbsession
l = s.query(xbxr.RunSession).order_by(xbxr.RunSession.timestamp.desc())
print([i.timestamp for i in l])

[datetime.datetime(2015, 4, 23, 21, 47, 28, 380382), datetime.datetime(2015, 4, 23, 21, 26, 16, 946288), datetime.datetime(2015, 4, 23, 18, 2, 52, 424848), datetime.datetime(2015, 4, 23, 16, 23, 9, 976050)]


Print latest RunSession

In [3]:
rs=l.first()
print(rs)

({'build_session_id': 6, 'config_hash': '138ef15dc44352639f075725a5383718e4fd7fbdd45bc2734c3434853a5a034d', 'host': 'zeus', 'id': 4, 'timestamp': datetime.datetime(2015, 4, 23, 21, 47, 28, 380382), 'xbh_mac': '001AB602D065', 'xbh_rev': 'e57b1b6', 'xbx_version': '64f5f13933a20c312b2ed5726e7f26953b7bf69b'}, ['drift_measurements', 'build_execs', 'build_session', 'config'])


We have overridden the `__repr__` function in the base class for SqlAlchemy tables to print out the type, a dictionary of contents, and a list of relations.

Let's print out all build executions:

In [4]:
print([i for i in rs.build_execs])

[({'build_id': 20, 'id': 6, 'run_session_id': 4, 'test_ok': True}, ['build', 'runs', 'run_session']), ({'build_id': 21, 'id': 8, 'run_session_id': 4, 'test_ok': False}, ['build', 'runs', 'run_session']), ({'build_id': 22, 'id': 7, 'run_session_id': 4, 'test_ok': False}, ['build', 'runs', 'run_session']), ({'build_id': 23, 'id': 9, 'run_session_id': 4, 'test_ok': True}, ['build', 'runs', 'run_session'])]


Not much information here. Let's print out information on the builds associated with the build executions:

In [5]:
print([(i, i.build) for i in rs.build_execs])

[(({'build_id': 20, 'id': 6, 'run_session_id': 4, 'test_ok': True}, ['build', 'runs', 'run_session']), ({'bss': 5320, 'build_ok': True, 'build_session_id': 6, 'compiler_idx': 0, 'data': 0, 'exe_path': 'ek-tm4c123gxl_16mhz/crypto_aead/0cipher/empty/0/xbdprog.bin', 'hex_checksum': '543c6c94056482ac551f46579b6c185a2ee2e3b33fe8d08af334e71c84f44a12', 'hex_path': 'ek-tm4c123gxl_16mhz/crypto_aead/0cipher/empty/0/xbdprog.hex', 'id': 20, 'implementation_hash': '1709ac0c04e3657b91bce5f4c570627b89fc1cfd6c943f950b8cade256ef05ec', 'log': b'LINK\nPOSTLINK\n', 'operation_name': 'crypto_aead', 'parallel_make': False, 'platform_hash': '4315375386478985adb47e513968dc493b5731b2b8c24c1f91ef6abbc146b4e7', 'primitive_name': '0cipher', 'rebuilt': True, 'text': 14380, 'timestamp': datetime.datetime(2015, 4, 23, 21, 47, 18, 61548), 'work_path': 'ek-tm4c123gxl_16mhz/crypto_aead/0cipher/empty/0'}, ['platform', 'compiler', 'operation', 'primitive', 'implementation', 'build_session'])), (({'build_id': 21, 'id': 8,

Not very readable. Let's use prettyprint:

In [6]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint([(i, i.build) for i in rs.build_execs])

[   (   ({'build_id': 20, 'id': 6, 'run_session_id': 4, 'test_ok': True}, ['build', 'runs', 'run_session']),
        ({'bss': 5320, 'build_ok': True, 'build_session_id': 6, 'compiler_idx': 0, 'data': 0, 'exe_path': 'ek-tm4c123gxl_16mhz/crypto_aead/0cipher/empty/0/xbdprog.bin', 'hex_checksum': '543c6c94056482ac551f46579b6c185a2ee2e3b33fe8d08af334e71c84f44a12', 'hex_path': 'ek-tm4c123gxl_16mhz/crypto_aead/0cipher/empty/0/xbdprog.hex', 'id': 20, 'implementation_hash': '1709ac0c04e3657b91bce5f4c570627b89fc1cfd6c943f950b8cade256ef05ec', 'log': b'LINK\nPOSTLINK\n', 'operation_name': 'crypto_aead', 'parallel_make': False, 'platform_hash': '4315375386478985adb47e513968dc493b5731b2b8c24c1f91ef6abbc146b4e7', 'primitive_name': '0cipher', 'rebuilt': True, 'text': 14380, 'timestamp': datetime.datetime(2015, 4, 23, 21, 47, 18, 61548), 'work_path': 'ek-tm4c123gxl_16mhz/crypto_aead/0cipher/empty/0'}, ['platform', 'compiler', 'operation', 'primitive', 'implementation', 'build_session'])),
    (   ({'bu

Better, but not good. The overridden repr implementation is supposed to be `eval`able. Note that we need to import datetime and call repr explicitly. Let's try:

In [7]:
import datetime
pp.pprint([(eval(repr(i)), eval(repr(i.build))) for i in rs.build_execs])

[   (   (   {'build_id': 20, 'id': 6, 'run_session_id': 4, 'test_ok': True},
            ['build', 'runs', 'run_session']),
        (   {   'bss': 5320,
                'build_ok': True,
                'build_session_id': 6,
                'compiler_idx': 0,
                'data': 0,
                'exe_path': 'ek-tm4c123gxl_16mhz/crypto_aead/0cipher/empty/0/xbdprog.bin',
                'hex_checksum': '543c6c94056482ac551f46579b6c185a2ee2e3b33fe8d08af334e71c84f44a12',
                'hex_path': 'ek-tm4c123gxl_16mhz/crypto_aead/0cipher/empty/0/xbdprog.hex',
                'id': 20,
                'implementation_hash': '1709ac0c04e3657b91bce5f4c570627b89fc1cfd6c943f950b8cade256ef05ec',
                'log': b'LINK\nPOSTLINK\n',
                'operation_name': 'crypto_aead',
                'parallel_make': False,
                'platform_hash': '4315375386478985adb47e513968dc493b5731b2b8c24c1f91ef6abbc146b4e7',
                'primitive_name': '0cipher',
                'r

Much better. We can see the 0hash and icepole implementations succeeded but the 0hash implementation failed. The log is mostly empty since we've rebuilt and thus there's not much makefile output. 

We want to know why the aesgcm implementations failed. To do this, we must examine the runs relation.

In [8]:
import datetime
for i in rs.build_execs:
    if not i.test_ok:
        for j in i.runs:
            print(j)

({'avg_power': None, 'build_exec_id': 8, 'id': 941, 'max_power': None, 'measured_cycles': None, 'median_power': None, 'min_power': None, 'reported_cycles': None, 'run_type': 'test_run', 'stack_usage': None, 'time': None, 'timestamp': None, 'total_energy': None}, ['power_samples', 'build_exec'])
({'avg_power': None, 'build_exec_id': 7, 'id': 940, 'max_power': None, 'measured_cycles': None, 'median_power': None, 'min_power': None, 'reported_cycles': None, 'run_type': 'test_run', 'stack_usage': None, 'time': None, 'timestamp': None, 'total_energy': None}, ['power_samples', 'build_exec'])


Too much stuff. Let's clean it up:

In [9]:
import datetime
for i in rs.build_execs:
    if not i.test_ok:
        for j in i.runs:
            print('{}: {}'.format(i.build.buildid,j.checksumfail_cause))

crypto_aead/aes256gcmv1/ref/0: Test failure: crypto_aead_encrypt does not handle a=c overlap
crypto_aead/aes128gcmv1/ref/0: Test failure: crypto_aead_encrypt does not handle a=c overlap
